In [1]:
import pandas as pd
import numpy as np
import json

In [3]:
train_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/train_hypothesis_evidences.csv')
train_df

,hypothesis,evidences,label
0,All the primary trial participants do not rece...,"[""INTERVENTION 1: "", "" Diagnostic (FLT PET)"",...",Contradiction
1,"Patients with Platelet count over 100,000/mm¬¨...","["" PATIENT CHARACTERISTICS:"", "" ANC 1,500/m...",Contradiction
2,Heart-related adverse events were recorded in ...,"[""Adverse Events 1:"", "" Supraventricular tach...",Entailment
3,Adult Patients with histologic confirmation of...,"[""Inclusion Criteria:"", "" Patients with histo...",Contradiction
4,Laser Therapy is in each cohort of the primary...,"[""INTERVENTION 1: "", "" Laser Therapy Alone"", ...",Contradiction
...,...,...,...
1695,"Adequate blood, kidney, and hepatic function a...","[""Inclusion Criteria:"", "" Postmenopausal wome...",Entailment
1696,The Ridaforolimus + Dalotuzumab + Exemestane g...,"[""Outcome Measurement: "", "" 1. Progression-fr...",Contradiction
1697,The only difference between the interventions ...,"[""INTERVENTION 1: "", "" Prone"", ""Prone positio...",Entailment
1698,Patients must have a white blood cell count ab...,"["" WBC > 1,500/mm\u00b3""]",Entailment


In [4]:
hypothesis_lst=train_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=train_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=train_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

1700

In [5]:
import torch
import numpy as np
import random
import math
from torch.utils.data import Dataset

class InputSequence(Dataset):
    def __init__(self, tok, l_text, l_text2, l_label=None, batch_size=64, gpu=True):
        self.data_len = len(l_text)
        self.data_idx = list(range(self.data_len))
        self.texts = tok(l_text, l_text2, padding=True, truncation=True, max_length=512, return_tensors='pt')
        self.l_label = np.array(l_label) if l_label is not None else None  # Initialize labels only if provided
        self.batch_size = batch_size
        self.gpu = gpu
        print('Tokenization done')

    def on_epoch_end(self):
        random.shuffle(self.data_idx)

    def __getitem__(self, i):
        start = i * self.batch_size
        batch_idx = self.data_idx[start:min(start + self.batch_size, self.data_len)]

        # Convert tensors to CUDA if GPU is enabled
        return_texts = {k: self.texts[k][batch_idx].cuda() if self.gpu else self.texts[k][batch_idx] for k in self.texts}

        if self.l_label is not None:  # Check if labels were provided
            return_labels = torch.from_numpy(self.l_label[batch_idx].astype(np.int64))
            if self.gpu:
                return_labels = return_labels.cuda()
            return return_texts, return_labels
        else:
            return return_texts  # Only return texts if no labels are available

    def __len__(self):
        return math.ceil(self.data_len / self.batch_size)


In [6]:
import random
import math
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

text_tok = AutoTokenizer.from_pretrained("domenicrosati/debertav3small-NLI4CT")
text_clf = AutoModelForSequenceClassification.from_pretrained("domenicrosati/debertav3small-NLI4CT")
text_clf.num_parameters()

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/881 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/283M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


70830722

In [8]:
training_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)
len(training_data)

Tokenization done


27

In [10]:
dev_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/dev_hypothesis_evidences.csv')
dev_df

,hypothesis,evidences,label
0,there is a 13.2% difference between the result...,"[""Outcome Measurement: "", "" Event-free Surviv...",Contradiction
1,Patients with significantly elevated ejection ...,"["" Cardiac left ventricular function with res...",Contradiction
2,a significant number of the participants in th...,"["" Enterocolitis 1/167 (0.60%)"", "" Enterocol...",Contradiction
3,the primary trial does not report the PFS or o...,"[""Outcome Measurement: "", "" Local Control Usi...",Entailment
4,Prior treatment with fulvestrant or with a pho...,"["" Prior treatment with a phosphatidylinosito...",Contradiction
...,...,...,...
195,The the primary trial intervention involves on...,"[""INTERVENTION 1: "", "" Letrozole"", "" Partici...",Contradiction
196,the secondary trial reported 1 single case of ...,"[""Adverse Events 1:"", "" Total: 16/48 (33.33%)...",Entailment
197,the secondary trial and the primary trial do n...,"[""Outcome Measurement: "", "" Number of Patient...",Entailment
198,the outcome measurement of the primary trial i...,"[""Outcome Measurement: "", "" Progression-free ...",Entailment


In [11]:
hypothesis_lst=dev_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=dev_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=dev_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

200

In [12]:
dev_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

Tokenization done


In [13]:
class Model(nn.Module):
    def __init__(self,clf):
        super(Model, self).__init__()
        self.clf=clf
        self.loss=nn.CrossEntropyLoss()

    def forward(self, texts, labels, gpu=True):

        loss=self.loss(self.clf(**texts).logits, labels)

        return loss

In [14]:
model=Model(text_clf)

In [15]:
import os
import shutil
import torch
from torch import nn
from transformers import AdamW
import numpy as np
import math
import random

bat_s = 16
l_rate = 1e-5

training_data.batch_size = bat_s

model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=l_rate)
total_epoch_num = 10

for epoch in range(total_epoch_num):
    training_data.on_epoch_end()
    loss_sum = 0.0
    loss_count = 0
    model.train()  # Ensure model is in training mode
    
    for batch in range(len(training_data)):
        optimizer.zero_grad()
        batch_texts, batch_labels = training_data[batch]
        loss_count += len(batch_texts['input_ids'])
        loss = model(batch_texts, batch_labels)
        print('epoch:', epoch, 'batch:', batch, 'loss:', loss.item(), end='\n' if batch == 0 or batch + 1 == len(training_data) or (batch + 1) % 1000 == 0 else '\r')
        loss_sum += loss.item() * len(batch_texts['input_ids'])
        loss.backward()
        optimizer.step()
        
    # Validation phase
    model.eval()  # Set model to evaluation mode
    val_loss_sum = 0.0
    val_loss_count = 0
    with torch.no_grad():
        for batch in range(len(dev_data)):
            batch_texts, batch_labels = dev_data[batch]
            val_loss_count += len(batch_texts['input_ids'])
            val_loss = model(batch_texts, batch_labels)
            val_loss_sum += val_loss.item() * len(batch_texts['input_ids'])

    avg_val_loss = val_loss_sum / val_loss_count
    print(f'Validation - Epoch: {epoch}, Avg Loss: {avg_val_loss}')

    # Save the model after each epoch
    model_path = f'./clf_models/domenicrosati/debertav3small-NLI4CT_epoch_{epoch:05d}.pt'
    model.clf.save_pretrained(model_path)
    print(f'Model saved to {model_path}')

_ = model.cpu()  # Move model back to CPU if necessary

epoch: 0 batch: 0 loss: 0.6781760454177856
epoch: 0 batch: 106 loss: 0.6895737648010254
Validation - Epoch: 0, Avg Loss: 0.6919023871421814
Model saved to ./clf_models/domenicrosati/debertav3small-NLI4CT_epoch_00000.pt
epoch: 1 batch: 0 loss: 0.7078342437744141
epoch: 1 batch: 106 loss: 0.6799120903015137
Validation - Epoch: 1, Avg Loss: 0.69091712474823
Model saved to ./clf_models/domenicrosati/debertav3small-NLI4CT_epoch_00001.pt
epoch: 2 batch: 0 loss: 0.7042086720466614
epoch: 2 batch: 106 loss: 0.5861951708793645
Validation - Epoch: 2, Avg Loss: 0.6834443402290344
Model saved to ./clf_models/domenicrosati/debertav3small-NLI4CT_epoch_00002.pt
epoch: 3 batch: 0 loss: 0.7044707536697388
epoch: 3 batch: 106 loss: 0.6784394383430481
Validation - Epoch: 3, Avg Loss: 0.6786336946487427
Model saved to ./clf_models/domenicrosati/debertav3small-NLI4CT_epoch_00003.pt
epoch: 4 batch: 0 loss: 0.6936289072036743
epoch: 4 batch: 106 loss: 0.6757081747055054
Validation - Epoch: 4, Avg Loss: 0.680

In [16]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['domenicrosati/debertav3small-NLI4CT'] + [
    '/kaggle/working/clf_models/domenicrosati/debertav3small-NLI4CT_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(dev_data)):
            batch_texts, batch_labels = dev_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

In [17]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.576704,0.668896,0.502513,1.00,0.505,0.484375,0.515625
1,1,0.581581,0.517766,0.525773,0.51,0.525,0.468750,0.531250
2,2,0.591533,0.562814,0.565657,0.56,0.565,0.445312,0.554688
3,3,0.617825,0.465116,0.555556,0.40,0.540,0.484375,0.515625
4,4,0.592344,0.628019,0.607477,0.65,0.615,0.406250,0.593750
5,5,0.602748,0.626728,0.581197,0.68,0.595,0.398438,0.601562
6,6,0.638774,0.645455,0.591667,0.71,0.610,0.437500,0.562500
7,7,0.631974,0.660793,0.590551,0.75,0.615,0.414062,0.585938
8,8,0.630715,0.676157,0.524862,0.95,0.545,0.437500,0.562500
9,9,0.688712,0.675214,0.589552,0.79,0.620,0.398438,0.601562


## Above results are for dev data

In [21]:
numerical_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/Numerical_Statements_hypothesis_evidences.csv')
print(numerical_df)

hypothesis_lst=numerical_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=numerical_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=numerical_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

numerical_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

                                           hypothesis  \
0   there is a 13.2% difference between the result...   
1   Patients with significantly elevated ejection ...   
2   Prior treatment with fulvestrant or with a pho...   
3   only patients with a HER2-positive status can ...   
4   the shortest PFS in cohort 1 of the primary tr...   
..                                                ...   
88  All the primary trial patients achieved either...   
89  A patient with a node positive T2 N2 M0 adenoc...   
90  the primary trial results show that the Trastu...   
91  the secondary trial reported 1 single case of ...   
92  All the primary trial patients had a minimum o...   

                                            evidences          label  
0   ["Outcome Measurement: ", "  Event-free Surviv...  Contradiction  
1   ["  Cardiac left ventricular function with res...  Contradiction  
2   ["  Prior treatment with a phosphatidylinosito...  Contradiction  
3   ["  HER2-positive status (p

In [22]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['domenicrosati/debertav3small-NLI4CT'] + [
    '/kaggle/working/clf_models/domenicrosati/debertav3small-NLI4CT_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(numerical_data)):
            batch_texts, batch_labels = numerical_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

In [23]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.475739,0.622222,0.451613,1.000000,0.451613,0.515625,0.484375
1,1,0.480273,0.450000,0.473684,0.428571,0.526882,0.562500,0.437500
2,2,0.498209,0.493827,0.512821,0.476190,0.559140,0.515625,0.484375
3,3,0.511628,0.405797,0.518519,0.333333,0.559140,0.484375,0.515625
4,4,0.498285,0.547619,0.547619,0.547619,0.591398,0.437500,0.562500
5,5,0.508101,0.539326,0.510638,0.571429,0.559140,0.484375,0.515625
6,6,0.513705,0.477273,0.456522,0.500000,0.505376,0.546875,0.453125
7,7,0.522137,0.527473,0.489796,0.571429,0.537634,0.500000,0.500000
8,8,0.489551,0.617886,0.469136,0.904762,0.494624,0.562500,0.437500
9,9,0.555915,0.568421,0.509434,0.642857,0.559140,0.531250,0.468750


## Above results are for numerical data

In [24]:
non_numerical_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/Non_Numerical_Statements_hypothesis_evidences.csv')
print(non_numerical_df)

hypothesis_lst=non_numerical_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=non_numerical_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=non_numerical_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

non_numerical_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

                                            hypothesis  \
0    a significant number of the participants in th...   
1    the primary trial does not report the PFS or o...   
2    the outcome measurement of the primary trial i...   
3    The the primary trial results section reports ...   
4    both cohorts of the primary trial receive iden...   
..                                                 ...   
102  Patients must be bedbound or severaly disabled...   
103  the results of the secondary trial and the pri...   
104  The the primary trial intervention involves on...   
105  the secondary trial and the primary trial do n...   
106  the outcome measurement of the primary trial i...   

                                             evidences          label  
0    ["  Enterocolitis 1/167 (0.60%)", "  Enterocol...  Contradiction  
1    ["Outcome Measurement: ", "  Local Control Usi...     Entailment  
2    ["Outcome Measurement: ", "  Progression-free ...  Contradiction  
3    ["Outcome 

In [25]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['domenicrosati/debertav3small-NLI4CT'] + [
    '/kaggle/working/clf_models/domenicrosati/debertav3small-NLI4CT_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(non_numerical_data)):
            batch_texts, batch_labels = non_numerical_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

In [26]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.657996,0.707317,0.547170,1.000000,0.551402,0.466667,0.533333
1,1,0.648592,0.564103,0.559322,0.568966,0.523364,0.466667,0.533333
2,2,0.656108,0.610169,0.600000,0.620690,0.570093,0.506667,0.493333
3,3,0.691570,0.504854,0.577778,0.448276,0.523364,0.480000,0.520000
4,4,0.667290,0.682927,0.646154,0.724138,0.635514,0.333333,0.666667
5,5,0.671612,0.687500,0.628571,0.758621,0.626168,0.413333,0.586667
6,6,0.725122,0.757576,0.675676,0.862069,0.700935,0.400000,0.600000
7,7,0.700590,0.750000,0.653846,0.879310,0.682243,0.360000,0.640000
8,8,0.731958,0.721519,0.570000,0.982759,0.588785,0.426667,0.573333
9,9,0.776497,0.748201,0.641975,0.896552,0.672897,0.373333,0.626667


## Above results are for Non numerical data

In [27]:
single_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/Single_hypothesis_evidences.csv')
print(single_df)

hypothesis_lst=single_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=single_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=single_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

single_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

                                            hypothesis  \
0    there is a 13.2% difference between the result...   
1    the primary trial does not report the PFS or o...   
2    Prior treatment with fulvestrant or with a pho...   
3    only patients with a HER2-positive status can ...   
4    the shortest PFS in cohort 1 of the primary tr...   
..                                                 ...   
135  A patient with a node positive T2 N2 M0 adenoc...   
136  Patients must be bedbound or severaly disabled...   
137  the primary trial results show that the Trastu...   
138  the outcome measurement of the primary trial i...   
139  All the primary trial patients had a minimum o...   

                                             evidences          label  
0    ["Outcome Measurement: ", "  Event-free Surviv...  Contradiction  
1    ["Outcome Measurement: ", "  Local Control Usi...     Entailment  
2    ["  Prior treatment with a phosphatidylinosito...  Contradiction  
3    ["  HER2-p

In [28]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['domenicrosati/debertav3small-NLI4CT'] + [
    '/kaggle/working/clf_models/domenicrosati/debertav3small-NLI4CT_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(single_data)):
            batch_texts, batch_labels = single_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

In [29]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.588805,0.669856,0.503597,1.000000,0.507143,0.468750,0.531250
1,1,0.597621,0.565789,0.524390,0.614286,0.528571,0.520833,0.479167
2,2,0.608444,0.601307,0.554217,0.657143,0.564286,0.500000,0.500000
3,3,0.625110,0.511278,0.539683,0.485714,0.535714,0.562500,0.437500
4,4,0.549715,0.623377,0.571429,0.685714,0.585714,0.531250,0.468750
5,5,0.577509,0.628205,0.569767,0.700000,0.585714,0.489583,0.510417
6,6,0.602343,0.621118,0.549451,0.714286,0.564286,0.520833,0.479167
7,7,0.597511,0.645570,0.579545,0.728571,0.600000,0.520833,0.479167
8,8,0.571835,0.666667,0.515625,0.942857,0.528571,0.489583,0.510417
9,9,0.645817,0.666667,0.564356,0.814286,0.592857,0.479167,0.520833


## Above results are for Single Type data

In [30]:
comparison_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/Comparison_hypothesis_evidences.csv')
print(comparison_df)

hypothesis_lst=comparison_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=comparison_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=comparison_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

comparison_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

                                           hypothesis  \
0   Patients with significantly elevated ejection ...   
1   a significant number of the participants in th...   
2   Diarrhoea is the most common adverse event in ...   
3   most participants in the secondary trial and t...   
4   Presence of hot flashes for at least 4 month p...   
5   the secondary trial and the primary trial both...   
6   the primary trial and the secondary trial use ...   
7   the primary trial and the secondary trial do n...   
8   the secondary trial and the primary trial do n...   
9   Between the secondary trial and the primary tr...   
10  All patients in the primary trial receive high...   
11  all participants of the primary trial have sta...   
12  The results from the secondary trial and the p...   
13  the primary trial and the secondary trial part...   
14  There are more cases of Urosepsis in the prima...   
15  the primary trial participants are not adminis...   
16  All the primary trial parti

In [31]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['domenicrosati/debertav3small-NLI4CT'] + [
    '/kaggle/working/clf_models/domenicrosati/debertav3small-NLI4CT_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(comparison_data)):
            batch_texts, batch_labels = comparison_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

In [32]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.605514,0.666667,0.500000,1.000000,0.500000,0.500000,0.500000
1,1,0.615455,0.355556,0.533333,0.266667,0.516667,0.483333,0.516667
2,2,0.629073,0.434783,0.625000,0.333333,0.566667,0.433333,0.566667
3,3,0.668129,0.307692,0.666667,0.200000,0.550000,0.450000,0.550000
4,4,0.712278,0.641509,0.739130,0.566667,0.683333,0.316667,0.683333
5,5,0.670585,0.622951,0.612903,0.633333,0.616667,0.383333,0.616667
6,6,0.738385,0.711864,0.724138,0.700000,0.716667,0.283333,0.716667
7,7,0.725717,0.695652,0.615385,0.800000,0.650000,0.350000,0.650000
8,8,0.752516,0.698795,0.547170,0.966667,0.583333,0.416667,0.583333
9,9,0.785449,0.698413,0.666667,0.733333,0.683333,0.316667,0.683333


## Above results are for Comparison Type data

In [34]:
adverse_events_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/AdverseEvents_hypothesis_evidences.csv')
print(adverse_events_df)

hypothesis_lst=adverse_events_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=adverse_events_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=adverse_events_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

adverse_events_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

                                           hypothesis  \
0   a significant number of the participants in th...   
1   The only case of congestive heart failure in t...   
2   Anaemia was the most common adverse event in c...   
3   The adverse events section in the primary tria...   
4   Diarrhoea is the most common adverse event in ...   
5   most participants in the secondary trial and t...   
6   The adverse events section in the primary tria...   
7   there were 4 types of Adverse events which did...   
8   None of the individual Aes recorded in the pri...   
9   The most common adverse event in the primary t...   
10  Febrile Neutropenia was the most common advers...   
11  None of the individual Aes recorded in the pri...   
12  the primary trial and the secondary trial do n...   
13  Between the secondary trial and the primary tr...   
14  Syncope was the most common adverse event reco...   
15  no cardiac or neural adverse events were recor...   
16  there was a higher percenta

In [35]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['domenicrosati/debertav3small-NLI4CT'] + [
    '/kaggle/working/clf_models/domenicrosati/debertav3small-NLI4CT_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(adverse_events_data)):
            batch_texts, batch_labels = adverse_events_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

In [36]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.625016,0.666667,0.500000,1.000000,0.500000,0.500000,0.500000
1,1,0.613243,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
2,2,0.599345,0.549020,0.560000,0.538462,0.557692,0.442308,0.557692
3,3,0.637744,0.555556,0.535714,0.576923,0.538462,0.461538,0.538462
4,4,0.579671,0.676923,0.564103,0.846154,0.596154,0.403846,0.596154
5,5,0.592049,0.714286,0.568182,0.961538,0.615385,0.384615,0.615385
6,6,0.670627,0.634921,0.540541,0.769231,0.557692,0.442308,0.557692
7,7,0.590762,0.735294,0.595238,0.961538,0.653846,0.346154,0.653846
8,8,0.612087,0.702703,0.541667,1.000000,0.576923,0.423077,0.576923
9,9,0.737622,0.721311,0.628571,0.846154,0.673077,0.326923,0.673077


## Above results are for Adverse events data

In [37]:
results_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/Results_hypothesis_evidences.csv')
print(results_df)

hypothesis_lst=results_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=results_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=results_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

results_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

                                           hypothesis  \
0   there is a 13.2% difference between the result...   
1   the primary trial does not report the PFS or o...   
2   the shortest PFS in cohort 1 of the primary tr...   
3   the outcome measurement of the primary trial i...   
4   The the primary trial results section reports ...   
5   The the primary trial results section reports ...   
6   The shortest PFS in cohort 1 of the primary tr...   
7   the secondary trial and the primary trial both...   
8   there is a minimal difference between the resu...   
9   the primary trial and the secondary trial use ...   
10  at least 2 participants of the primary trial, ...   
11  the primary trial does not report the PFS or o...   
12  The results from the secondary trial and the p...   
13  neither of the interventions in the primary tr...   
14  The shortest PFS in cohort 1 of the primary tr...   
15  there were 6 recorded deaths in cohort 1 of th...   
16  the primary trial reports t

In [38]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['domenicrosati/debertav3small-NLI4CT'] + [
    '/kaggle/working/clf_models/domenicrosati/debertav3small-NLI4CT_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(results_data)):
            batch_texts, batch_labels = results_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

In [39]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.596894,0.666667,0.500000,1.000000,0.500000,0.500000,0.500000
1,1,0.598128,0.307692,0.545455,0.214286,0.517857,0.482143,0.517857
2,2,0.611060,0.390244,0.615385,0.285714,0.553571,0.446429,0.553571
3,3,0.634482,0.324324,0.666667,0.214286,0.553571,0.446429,0.553571
4,4,0.629262,0.533333,0.705882,0.428571,0.625000,0.375000,0.625000
5,5,0.615404,0.509804,0.565217,0.464286,0.553571,0.446429,0.553571
6,6,0.698702,0.641509,0.680000,0.607143,0.660714,0.339286,0.660714
7,7,0.749633,0.620690,0.600000,0.642857,0.607143,0.392857,0.607143
8,8,0.708475,0.666667,0.545455,0.857143,0.571429,0.428571,0.571429
9,9,0.724572,0.581818,0.592593,0.571429,0.589286,0.410714,0.589286


## Above results are for Results data

In [41]:
eligibility_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/Eligibility_hypothesis_evidences.csv')
print(eligibility_df)

hypothesis_lst=eligibility_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=eligibility_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=eligibility_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

eligibility_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

                                           hypothesis  \
0   Patients with significantly elevated ejection ...   
1   Prior treatment with fulvestrant or with a pho...   
2   only patients with a HER2-positive status can ...   
3   T4 N2 M4 patients are eligible for the primary...   
4   Patients must have a palpable carcinoma to be ...   
5   to be eligible for the primary trial patients ...   
6   ongoing flucytosine treatments are not permitt...   
7   Only patients with triple negative breast canc...   
8   adults and children can take part in the prima...   
9   Prior treatment with fulvestrant or with a pho...   
10  Presence of hot flashes for at least 4 month p...   
11  to be eligible for the primary trial candidate...   
12  A patient with a node positive T2 N2 M0 adenoc...   
13  ECOG score < 2 is necessary to be eligible for...   
14  patients with a Malignant brain tumour or colo...   
15  the secondary trial and the primary trial do n...   
16  all participants of the pri

In [42]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['domenicrosati/debertav3small-NLI4CT'] + [
    '/kaggle/working/clf_models/domenicrosati/debertav3small-NLI4CT_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(eligibility_data)):
            batch_texts, batch_labels = eligibility_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

In [43]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.624653,0.674699,0.509091,1.000000,0.517857,0.482143,0.517857
1,1,0.659345,0.648649,0.521739,0.857143,0.535714,0.464286,0.535714
2,2,0.665264,0.647887,0.534884,0.821429,0.553571,0.446429,0.553571
3,3,0.689735,0.526316,0.517241,0.535714,0.517857,0.482143,0.517857
4,4,0.573968,0.610169,0.580645,0.642857,0.589286,0.410714,0.589286
5,5,0.677969,0.655172,0.633333,0.678571,0.642857,0.357143,0.642857
6,6,0.686082,0.666667,0.600000,0.750000,0.625000,0.375000,0.625000
7,7,0.678201,0.642857,0.642857,0.642857,0.642857,0.357143,0.642857
8,8,0.648722,0.674699,0.509091,1.000000,0.517857,0.482143,0.517857
9,9,0.640350,0.685714,0.571429,0.857143,0.607143,0.392857,0.607143


## Above results are for Eligibility data

In [44]:
intervention_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/Intervention_hypothesis_evidences.csv')
print(intervention_df)

hypothesis_lst=intervention_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=intervention_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=intervention_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

intervention_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

                                           hypothesis  \
0   Both cohorts of the primary trial undergo a to...   
1   both cohorts of the primary trial receive iden...   
2   the primary trial participants recieve doses o...   
3   Cohort 1 and 2 of the primary trial do not rec...   
4   Participants of the primary trial will not rec...   
5   the primary trial participants receive more th...   
6   All patients in the primary trial receive high...   
7   the primary trial participants are administere...   
8   the primary trial and the secondary trial part...   
9   Cohort 1 and 2 of the primary trial do not rec...   
10  the primary trial participants are not adminis...   
11  All the primary trial participants are adminis...   
12  the primary trial participants are administere...   
13  All the primary trial participants are adminis...   
14  the primary trial participants are administere...   
15  80% of patients in the primary trial received ...   
16  the primary trial participa

In [45]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['domenicrosati/debertav3small-NLI4CT'] + [
    '/kaggle/working/clf_models/domenicrosati/debertav3small-NLI4CT_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(intervention_data)):
            batch_texts, batch_labels = intervention_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

In [46]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.588996,0.666667,0.500000,1.000000,0.500000,0.500000,0.500000
1,1,0.592892,0.500000,0.571429,0.444444,0.555556,0.444444,0.555556
2,2,0.574934,0.611111,0.611111,0.611111,0.611111,0.388889,0.611111
3,3,0.646647,0.333333,0.666667,0.222222,0.555556,0.444444,0.555556
4,4,0.613669,0.684211,0.650000,0.722222,0.666667,0.333333,0.666667
5,5,0.578946,0.578947,0.550000,0.611111,0.555556,0.444444,0.555556
6,6,0.605632,0.634146,0.565217,0.722222,0.583333,0.416667,0.583333
7,7,0.587992,0.622222,0.518519,0.777778,0.527778,0.472222,0.527778
8,8,0.602958,0.653846,0.500000,0.944444,0.500000,0.500000,0.500000
9,9,0.674854,0.708333,0.566667,0.944444,0.611111,0.388889,0.611111


## Above results are for Intervention data